
# PROs preprocess:


received the raw data file from Sophia, who did the first step of preprocessing


Yiyu


In [20]:
%conda env list


# conda environments:
#
base                     /Users/yiyuwang/anaconda3
actigraphy               /Users/yiyuwang/anaconda3/envs/actigraphy
flywheel                 /Users/yiyuwang/anaconda3/envs/flywheel
neuroimaging_analysis     /Users/yiyuwang/anaconda3/envs/neuroimaging_analysis
prediction_env        *  /Users/yiyuwang/anaconda3/envs/prediction_env
stabl                    /Users/yiyuwang/anaconda3/envs/stabl
                         /Users/yiyuwang/fsl


Note: you may need to restart the kernel to use updated packages.


In [21]:
import pandas as pd
import glob, sys
import numpy as np
import os


from sklearn import metrics
import warnings
import copy

import re
import math
import time
import glob
import random
import string
import collections
import seaborn as sns

import matplotlib.pyplot as plt

SEED = 2024
NROI = 100

In [22]:
# local directory
project_dir = '/users/yiyuwang/Desktop/SNAPL/Projects/HEAL_prediction'

# # directory on sherlock
# project_dir = '/scratch/users/yiyuw/Projects/HEAL_prediction'



In [23]:
# calculate reliability between two runs:

warnings.filterwarnings('ignore')

    
def FormatSubID(source_id):
    # Make it a string
    source_id = str(source_id)
    if "bio" in source_id:   
        # check if the sub number is four digit:
        sub_num = int(source_id.split("bio")[1])
        formatted_id = "bio" + "{:04}".format(sub_num)
        
    elif "Bio" in source_id:
        # check if the sub number is four digit:
        sub_num = int(source_id.split("Bio")[1])
        formatted_id = "bio" + "{:04}".format(sub_num)
        
    elif "BIO-" in source_id:
        # check if the sub number is four digit:
        sub_num = int(source_id.split("BIO-")[1])
        formatted_id = "bio" + "{:04}".format(sub_num)
    elif "BIO" in source_id:
        # check if the sub number is four digit:
        sub_num = int(source_id.split("BIO")[1])
        formatted_id = "bio" + "{:04}".format(sub_num)    
        
    elif type(int(source_id))==int: #sub_id just numbers
        sub_num = int(source_id)
        formatted_id = "bio" + "{:04}".format(sub_num)
    
    else: # everything else just return empty string
        formatted_id = ""
    
    return formatted_id

def FormatSession(session):
    if 'baseline' in session:
        formatted_session = 'baseline'
    elif '1_month' in session:
        formatted_session = 'month1'
    elif '2_month' in session:
        formatted_session = 'month2'    
    elif '3_month' in session:
        formatted_session = 'month3'
    elif '6_month' in session:
        formatted_session = 'month6'
    else:
        formatted_session = ""
    return formatted_session        

def FormatSessionPromis(session):
    if session == 'b':
        formatted_session = 'baseline'
    elif session == '1':
        formatted_session = 'month1'
    elif session == '2':
        formatted_session = 'month2'    
    elif session == '3':
        formatted_session = 'month3'
    elif session == '6':
        formatted_session = 'month6'
    else:
        formatted_session = ""
    return formatted_session

In [24]:

pro_df = pd.read_csv(f'{project_dir}/Data/raw_data/PROs/BiomarkersPatientRep_DATA_2024-06-01_1201.csv')
pro_df.head()

,study_id,redcap_event_name,redcap_survey_identifier,survey_welcome_timestamp,date_survey_taken,survey_welcome_complete,demographics_timestamp,date_taken,brthdtc,age,...,fcrtracking_lastname,fcrtracking_mrn,id_screening,fcrtracking_dob,fcrtracking_age,fcrtracking_phone,fcrtracking_email,fcrtracking_city,fcrtracking_zip,identifiers_complete
0,Bio0001,baseline_arm_1,NaN,4/4/22 20:07,4/4/22,2.0,4/4/22 20:10,4/4/22,6/6/56,65.827498,...,NaN,NaN,NaN,NaN,NaN,NaN,janeanmike@yahoo.com,NaN,NaN,0.0
1,Bio0001,screening__enrollm_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Felch,30864458.0,NaN,6/6/56,66.0,(831) 521-2189,janeanmike@yahoo.com,San Jose,95123.0,2.0
2,Bio0001,1_month_followup_arm_1,NaN,5/5/22 12:18,5/5/22,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Bio0001,2_month_followup_arm_1,NaN,6/9/22 11:29,6/9/22,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Bio0001,3_month_followup_arm_1,NaN,6/30/22 11:08,6/30/22,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
pro_df = pd.read_csv(f'{project_dir}/Data/raw_data/PROs/BiomarkersPatientRep_DATA_2024-06-01_1201.csv')

pro_df['cpaq_8_rev'] =  8 - pro_df['cpaq_8']
pro_df['cpaq_7_rev'] = 7 - pro_df['cpaq_8']
pro_df['cpaq_6_rev'] = 6 - pro_df['cpaq_6']


score_df = pd.DataFrame({'participant_id': pro_df['study_id'], 'time': pro_df['redcap_event_name']})
score_df['participant_id'] = score_df['participant_id'].apply(FormatSubID)
score_df['time'] = score_df['time'].apply(FormatSession)
# if time or participant id is empty, remove the row
score_df = score_df[score_df['participant_id'] != ""]
score_df = score_df[score_df['time'] != ""]
score_df['cpaq_willingness_7_rev'] =  pro_df[['cpaq_' + str(i) for i in [2,4,7]]].sum(axis=1) + pro_df['cpaq_7_rev']
score_df['cpaq_willingness_8_rev'] = pro_df[['cpaq_' + str(i) for i in [2,4,7]]].sum(axis=1) + pro_df['cpaq_8_rev']
score_df['cpaq_willingness_6_rev'] = pro_df[['cpaq_' + str(i) for i in [2,4,7]]].sum(axis=1) + pro_df['cpaq_6_rev']
score_df['cpaq_willingness_norev']= pro_df[['cpaq_' + str(i) for i in [2,4,7, 8]]].sum(axis=1)

pro_df[['cpaq_8_rev', 'cpaq_7_rev', 'cpaq_6_rev', 'cpaq_8']].head()

score_df[['cpaq_willingness_7_rev', 'cpaq_willingness_8_rev', 'cpaq_willingness_6_rev', 'cpaq_willingness_norev', 'time', 'participant_id']].head()

,cpaq_willingness_7_rev,cpaq_willingness_8_rev,cpaq_willingness_6_rev,cpaq_willingness_norev,time,participant_id
0,19.0,20.0,18.0,12.0,baseline,bio0001
2,12.0,13.0,12.0,11.0,month1,bio0001
3,11.0,12.0,9.0,6.0,month2,bio0001
4,12.0,13.0,13.0,9.0,month3,bio0001
5,9.0,10.0,7.0,8.0,month6,bio0001


In [26]:
pro_df.columns

Index(['study_id', 'redcap_event_name', 'redcap_survey_identifier',
       'survey_welcome_timestamp', 'date_survey_taken',
       'survey_welcome_complete', 'demographics_timestamp', 'date_taken',
       'brthdtc', 'age',
       ...
       'fcrtracking_dob', 'fcrtracking_age', 'fcrtracking_phone',
       'fcrtracking_email', 'fcrtracking_city', 'fcrtracking_zip',
       'identifiers_complete', 'cpaq_8_rev', 'cpaq_7_rev', 'cpaq_6_rev'],
      dtype='object', length=696)

# scoring raw data

In [27]:
score_df = pd.DataFrame({'participant_id': pro_df['study_id'], 'time': pro_df['redcap_event_name']})
score_df['participant_id'] = score_df['participant_id'].apply(FormatSubID)
score_df['time'] = score_df['time'].apply(FormatSession)
# if time or participant id is empty, remove the row
score_df = score_df[score_df['participant_id'] != ""]
score_df = score_df[score_df['time'] != ""]


# Chronic Pain Acceptance Questionnaire
# reverse score item 8 (range = 0-6)
pro_df['cpaq_8'] = 6 - pro_df['cpaq_8']

cpaq_items = ['cpaq_' + str(i) for i in [1,2,3,4,5,6,7,8]]
# if all the items have been answered, sum the score
row_mask = pro_df[cpaq_items].apply(lambda row: row.isnull().sum() == 0, axis=1)

# replace False with nan such that it will return an nan if any of the items is missing
row_mask = row_mask.replace(False, np.nan)
# sum the score
score_df['cpaq_willingness'] = pro_df[['cpaq_' + str(i) for i in [2,4,7,8]]].sum(axis=1) * row_mask
score_df['cpaq_activity_engagement'] = pro_df[['cpaq_' + str(i) for i in [1,3,5,6]]].sum(axis=1) * row_mask
score_df['cpaq_total'] = score_df['cpaq_willingness'] + score_df['cpaq_activity_engagement']



# Pain Catastrophizing Scale
pcs_items = ['pcs_' + str(i) for i in range(1,14)]  
row_mask = pro_df[pcs_items].apply(lambda row: row.isnull().sum() == 0, axis=1)

# replace False with nan such that it will return an nan if any of the items is missing
row_mask = row_mask.replace(False, np.nan)

score_df['pcs_total'] = pro_df[pcs_items].sum(axis=1) * row_mask
score_df['pcs_rumination'] = pro_df[['pcs_8', 'pcs_9', 'pcs_10', 'pcs_11']].sum(axis=1) * row_mask
score_df['pcs_magnification'] = pro_df[['pcs_6', 'pcs_7', 'pcs_13']].sum(axis=1) * row_mask
score_df['pcs_hopelessness'] = pro_df[['pcs_1', 'pcs_2', 'pcs_3', 'pcs_4', 'pcs_5', 'pcs_12']].sum(axis=1) * row_mask


#  'scltotal', 'pseqtotal', 'gadtotal', 'phqtotal', 'tsktotal'
scl_items = ['scl90_' + str(i) for i in range(1,12)]  
row_mask = pro_df[scl_items].apply(lambda row: row.isnull().sum() == 0, axis=1)
row_mask = row_mask.replace(False, np.nan)
score_df['scltotal'] = pro_df[scl_items].sum(axis=1) * row_mask

pseq_items = ['pseq_' + str(i) for i in range(1,11)]
row_mask = pro_df[pseq_items].apply(lambda row: row.isnull().sum() == 0, axis=1)
row_mask = row_mask.replace(False, np.nan)
score_df['pseqtotal'] = pro_df[pseq_items].sum(axis=1) * row_mask

gad_items = ['gad_' + str(i) for i in [1,2]]
row_mask = pro_df[gad_items].apply(lambda row: row.isnull().sum() == 0, axis=1)
row_mask = row_mask.replace(False, np.nan)
score_df['gadtotal'] = pro_df[gad_items].sum(axis=1) * row_mask

tsk_items = ['tsk_' + str(i) for i in range(1,12)]
row_mask = pro_df[tsk_items].apply(lambda row: row.isnull().sum() == 0, axis=1)
row_mask = row_mask.replace(False, np.nan)
score_df['tsktotal'] = pro_df[tsk_items].sum(axis=1) * row_mask


# list of variables that can be taken directly
direct_variables = ['working_not_working', 'pain_frequency', 'life_limitation', 'describe_pain_number', 'phq_2',
                    'taps_1', 'taps_2', 'taps_3', 'taps_4',	'taps_5', 'ppiy_1', 'ppiy_2a', 'ppiy_3a']
score_df[direct_variables] = pro_df[direct_variables]







In [28]:
# score_df[['participant_id', 'time', 'cpaq_willingness', 'cpaq_total', 'cpaq_activity_engagement']].to_csv('cpaq.csv', index=False)

# sleep duration: preliminary processing

sleep duration will be scored by hand because open texts are not reliable

In [29]:
sleep_df = pro_df[['study_id', 'redcap_event_name', 'sleepduration']]
sleep_df['participant_id'] = sleep_df['study_id'].apply(FormatSubID)
sleep_df['time'] = sleep_df['redcap_event_name'].apply(FormatSession)
sleep_df = sleep_df[sleep_df['participant_id'] != ""]
sleep_df = sleep_df[sleep_df['time'] != ""]
sleep_df.to_csv(f'{project_dir}/Data/raw_data/PROs/sleep_duration.csv', index=False)


In [30]:
import pandas as pd
import numpy as np
import re

def clean_value(value):
    # Remove unwanted characters
    cleaned_value = re.sub(r'[^\d.-]', '', value)
    # Fix multiple periods issue
    if cleaned_value.count('.') > 1:
        parts = cleaned_value.split('.')
    
        cleaned_value = parts[0] + '.' + ''.join(parts[1:])
    return cleaned_value

def convert_to_numeric(value):
    if pd.isna(value):
        return np.nan
    if isinstance(value, pd.Timestamp):
        return np.nan
    if isinstance(value, str):
        value = value.lower().replace('hours', 'hr').replace('hour', 'hr').replace('minutes', 'min').replace('minute', 'min')
        if 'hr' in value and 'min' in value:
            try:
                hr_part = float(re.search(r'(\d+)\s*hr', value).group(1))
                min_part = float(re.search(r'(\d+)\s*min', value).group(1))
                return hr_part + (min_part / 60)
            except:
                return np.nan
        value = clean_value(value)
        if '-' in value:
            try:
                start, end = map(float, value.split('-'))
                return (start + end) / 2
            except ValueError:
                return np.nan
        elif value.replace('.', '', 1).isdigit():
            return float(value)
        else:
            return np.nan
    if isinstance(value, (int, float)):
        return float(value)
    return np.nan

# Load the Excel file

df = sleep_df

# Apply the conversion function to the dataframe
df['Converted'] = df['sleepduration'].apply(convert_to_numeric)

# replace with nan for anything that is longer than 24 hours
# df.loc[df['Converted'] > 24, 'Converted'] = np.nan

# Display the converted dataframe
print(df.head())


#!!!!!!!!!!! turn this to false once hand processed the data!!!!!!!!!!!!!!!!!!
save = False
if save:
    df.to_csv(f'{project_dir}/Data/raw_data/PROs/converted_sleep_duration.csv', index=False)
# df still need to be manually checked for errors

  study_id       redcap_event_name sleepduration participant_id      time  \
0  Bio0001          baseline_arm_1             7        bio0001  baseline   
2  Bio0001  1_month_followup_arm_1             6        bio0001    month1   
3  Bio0001  2_month_followup_arm_1             6        bio0001    month2   
4  Bio0001  3_month_followup_arm_1             6        bio0001    month3   
5  Bio0001  6_month_followup_arm_1             6        bio0001    month6   

   Converted  
0        7.0  
2        6.0  
3        6.0  
4        6.0  
5        6.0  


In [31]:
sleep_df = pd.read_csv(f'{project_dir}/Data/raw_data/PROs/converted_sleep_duration.csv')
# merge based on participant_id and time
score_df = score_df.merge(sleep_df[['participant_id', 'time', 'Converted']], on=['participant_id', 'time'], how='left')
score_df.rename(columns={'Converted': 'sleepduration'}, inplace=True)

score_df

,participant_id,time,cpaq_willingness,cpaq_activity_engagement,cpaq_total,pcs_total,pcs_rumination,pcs_magnification,pcs_hopelessness,scltotal,...,phq_2,taps_1,taps_2,taps_3,taps_4,taps_5,ppiy_1,ppiy_2a,ppiy_3a,sleepduration
0,bio0001,baseline,18.0,6.0,24.0,12.0,3.0,1.0,8.0,23.0,...,1.0,4.0,4.0,4.0,4.0,4.0,8.0,5.0,6.0,7.0
1,bio0001,month1,11.0,7.0,18.0,15.0,5.0,2.0,8.0,17.0,...,1.0,4.0,4.0,4.0,4.0,4.0,9.0,6.0,4.0,6.0
2,bio0001,month2,10.0,7.0,17.0,8.0,3.0,0.0,5.0,18.0,...,1.0,4.0,4.0,4.0,4.0,4.0,9.0,6.0,6.0,6.0
3,bio0001,month3,11.0,5.0,16.0,9.0,4.0,1.0,4.0,21.0,...,1.0,4.0,4.0,4.0,4.0,4.0,8.0,6.0,6.0,6.0
4,bio0001,month6,8.0,8.0,16.0,14.0,6.0,2.0,6.0,22.0,...,1.0,4.0,4.0,4.0,4.0,4.0,10.0,6.0,5.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1086,bio0335,baseline,14.0,20.0,34.0,9.0,4.0,3.0,2.0,11.0,...,1.0,4.0,4.0,4.0,4.0,4.0,6.0,5.0,5.0,7.5
1087,bio0337,baseline,10.0,21.0,31.0,13.0,4.0,3.0,6.0,1.0,...,1.0,4.0,4.0,4.0,4.0,4.0,4.0,2.0,1.0,5.5
1088,bio0338,baseline,8.0,24.0,32.0,4.0,1.0,2.0,1.0,7.0,...,0.0,4.0,4.0,4.0,4.0,4.0,3.0,2.0,2.0,6.0
1089,bio0339,baseline,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
score_df.columns

Index(['participant_id', 'time', 'cpaq_willingness',
       'cpaq_activity_engagement', 'cpaq_total', 'pcs_total', 'pcs_rumination',
       'pcs_magnification', 'pcs_hopelessness', 'scltotal', 'pseqtotal',
       'gadtotal', 'tsktotal', 'working_not_working', 'pain_frequency',
       'life_limitation', 'describe_pain_number', 'phq_2', 'taps_1', 'taps_2',
       'taps_3', 'taps_4', 'taps_5', 'ppiy_1', 'ppiy_2a', 'ppiy_3a',
       'sleepduration'],
      dtype='object')

# combining PROMIS

In [33]:
promis_only_score_df = pd.DataFrame({'participant_id': score_df['participant_id'], 'time': score_df['time']})
include_pi = True
if include_pi:

    promis_list = ['anger', 'anxiety', 'depression', 'painbe', 'sleep', 'fatigue', 'emotional', 'physical', 'satisfaction', 'social', 'sleepim', 'upperex', 'painin']
else:
    promis_list = ['anger', 'anxiety', 'depression', 'painbe', 'sleep', 'fatigue', 'emotional', 'physical', 'satisfaction', 'social', 'sleepim', 'upperex']

for promis in promis_list:
    promis_df = pd.read_csv(f'{project_dir}/Data/raw_data/PROs/{promis}.csv', skiprows=4)
    promis_df['participant_id'] = promis_df['PIN'].apply(FormatSubID)
    promis_df['time'] = promis_df['Assmnt'].apply(FormatSessionPromis)
    promis_df = promis_df[promis_df['participant_id'] != ""]
    promis_df = promis_df[promis_df['time'] != ""]
    
    score_df = score_df.merge(promis_df[['participant_id', 'time', 'TScore']], on=['participant_id', 'time'], how='left')
    score_df.rename(columns={'TScore': f'PROMIS-{promis}_T'}, inplace=True)

    promis_only_score_df = promis_only_score_df.merge(promis_df[['participant_id', 'time', 'TScore']], on=['participant_id', 'time'], how='left')
    promis_only_score_df.rename(columns={'TScore': f'PROMIS-{promis}_T'}, inplace=True)

promis_only_score_df.rename(columns={'participant_id': 'subject_id', 'time': 'session'}, inplace=True)    
promis_only_score_df.to_csv(f'{project_dir}/Data/cleaned_data/promis_only.csv', index=False)

# break into time points

target_dir = '/Volumes/Projects/HEAL_dataset_dev/'
score_df.rename(columns={'participant_id': 'subject_id', 'time': 'session'}, inplace=True)
for session in score_df['session'].unique():
    score_df[score_df['session']==session].to_csv(f'{target_dir}/ses-{session}/PROs/ses-{session}_desc-PRO.csv', index=False)

score_df.to_csv(f'{project_dir}/Data/cleaned_data/PROs_ses-all.csv', index=False)

##  save score_df

In [16]:
# break into time points

target_dir = '/Volumes/Projects/HEAL_dataset_dev/'
score_df.rename(columns={'participant_id': 'subject_id', 'time': 'session'}, inplace=True)
for session in score_df['session'].unique():
    score_df[score_df['session']==session].to_csv(f'{target_dir}/ses-{session}/PROs/ses-{session}_desc-PRO.csv', index=False)

score_df.to_csv(f'{project_dir}/Data/cleaned_data/PROs_ses-all.csv', index=False)

# save PainIn separately

In [17]:
promis = 'painin'
threshold = 65
promis_df = pd.read_csv(f'{project_dir}/Data/raw_data/PROs/{promis}.csv', skiprows=4)
target_dir = '/Volumes/Projects/HEAL_dataset_dev/'
promis_df['subject_id'] = promis_df['PIN'].apply(FormatSubID)
promis_df['session'] = promis_df['Assmnt'].apply(FormatSessionPromis)
promis_df = promis_df[promis_df['subject_id'] != ""]
promis_df = promis_df[promis_df['session'] != ""]
promis_df['group'] = promis_df['TScore'].apply(lambda x: 1 if x >= threshold else 0)
promis_df = promis_df[['subject_id', 'session', 'TScore', 'group']]
promis_df.to_csv(f'{project_dir}/Data/cleaned_data/PROs_{promis}_threshold-{threshold}.csv', index=False)
promis_df.to_csv(f'{project_dir}/labels.csv', index=False) # labels for prediction models
for session in promis_df['session'].unique():
    promis_df[promis_df['session']==session].to_csv(f'{target_dir}/ses-{session}/PROs/ses-{session}_desc-{promis}_threshold-{threshold}.csv', index=False)


In [18]:
project_dir

'/users/yiyuwang/Desktop/SNAPL/Projects/HEAL_prediction'

# preprocessing EEG, MRI parcellation, demographics

In [ ]:
def convert_to_year(number):
    number = str(number).split('/')[2]

    try:
        year = int(number)
    except ValueError:
        raise ValueError("Input must be a valid number")
    if year < 0 or year > 99:
        raise ValueError("Input must be between 00 and 99")
    if year < 25:
        return 2000 + year
    return 1900 + year



# Example usage
input_number = "6/8/02"
converted_number = convert_to_year(input_number)
print(converted_number)


# Kaisa provided the raw csv from oncore (accural_results.csv)
demographic_df = pd.read_csv(f'../Data/raw_data/PROS/demographics_oncore_jun2024.csv', skiprows=4).astype(str)
demographic_df['StudyYear'] = demographic_df['On Study Date'].apply(convert_to_year)
demographic_df['BirthYear'] =demographic_df['Birth Date'].apply(convert_to_year)

demographic_df

In [18]:
# concatenate all EEG apriori cluster

which_type = 'source'
which_source = 'aprioriCluster' # or which_source = ''
eeg_dir = f'/Volumes/mackeylab/Mackeylab/Individual_Folders/YiyuWang/HEAL_EEG/{which_type}/{which_source}/'

for session_id in range(5):
    if session_id == 0:
        df = pd.read_csv(eeg_dir + f'0{session_id}_mergedRECREOdB.csv')
        # replace "_REC_0{session_id}" with empty string in column names to keep column names consistent across sessions
        df.columns = [col.replace(f'_REC_0{session_id}', '') for col in df.columns]
        df['session'] = 'baseline'

    else:
        ses_df = pd.read_csv(eeg_dir + f'0{session_id}_mergedRECREOdB.csv')
        ses_df.columns = [col.replace(f'_REC_0{session_id}', '') for col in ses_df.columns]
        
        
        if session_id == 4:
            ses_df['session'] = 'month6'
        else:     
            ses_df['session'] = f'month{session_id}'
        df = pd.concat([df, ses_df])    
df.rename(columns={'subnum': 'subject_id'}, inplace=True)   
# move session to the second column
df = df[['subject_id', 'session'] + [col for col in df.columns if col != 'subject_id' and col != 'session']]
# zscore by row (not including the study_id and session column)
df.iloc[:, 2:] = df.iloc[:, 2:].apply(lambda row: (row - row.mean()) / row.std(), axis=1)
df.to_csv(f'HEAL_EEG_{which_type}_{which_source}.csv', index=False)   

In [19]:
df

,subject_id,session,REC2_1,REC2_2,REC2_3,REC2_4,REC2_5,REC2_6,REC2_7,REC2_8,...,REO2_9891,REO2_9892,REO2_9893,REO2_9894,REO2_9895,REO2_9896,REO2_9897,REO2_9898,REO2_9899,REO2_9900
0,bio0001,baseline,0.635529,0.577702,0.493721,0.390051,0.275858,0.160531,0.052460,-0.042753,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,bio0002,baseline,0.184276,0.077811,-0.019691,-0.103823,-0.171910,-0.223638,-0.260548,-0.285547,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,bio0003,baseline,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,bio0004,baseline,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,bio0005,baseline,-0.400284,-0.465622,-0.525068,-0.576268,-0.617913,-0.650042,-0.673683,-0.690568,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,bio0248,month6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132,bio0251,month6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
133,bio0258,month6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134,bio0259,month6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# concatenate all EEG

which_type = 'source'
which_source = 'aprioriCluster' # or which_source = ''
eeg_dir = f'/Volumes/mackeylab/Mackeylab/Individual_Folders/YiyuWang/HEAL_EEG/{which_type}/'

for session_id in range(5):
    if session_id == 0:
        if which_type == 'scalp':
            df = pd.read_csv(eeg_dir + f'mergedScalpEEG_REC_REO_0{session_id}.csv')
        elif which_type == 'source':   
            df = pd.read_csv(eeg_dir + f'mergedEEGIC_REC_REO_0{session_id}.csv')
        # replace "_REC_0{session_id}" with empty string in column names to keep column names consistent across sessions
        df.columns = [col.replace(f'_REC_0{session_id}', '') for col in df.columns]
        df['session'] = 'baseline'

    else:
        if which_type == 'scalp':
            ses_df = pd.read_csv(eeg_dir + f'mergedScalpEEG_REC_REO_0{session_id}.csv')
        elif which_type == 'source':   
            ses_df = pd.read_csv(eeg_dir + f'mergedEEGIC_REC_REO_0{session_id}.csv')
        
        ses_df.columns = [col.replace(f'_REC_0{session_id}', '') for col in ses_df.columns]
        
        
        if session_id == 4:
            ses_df['session'] = 'month6'
        else:     
            ses_df['session'] = f'month{session_id}'
        df = pd.concat([df, ses_df])    
df.rename(columns={'subNum': 'subject_id'}, inplace=True)   
# move session to the second column
df = df[['subject_id', 'session'] + [col for col in df.columns if col != 'subject_id' and col != 'session']]
# zscore by row (not including the study_id and session column)
df.iloc[:, 2:] = df.iloc[:, 2:].apply(lambda row: (row - row.mean()) / row.std(), axis=1)
df.to_csv(f'HEAL_EEG_{which_type}.csv', index=False)   

In [ ]:
# concatenate all EEG


eeg_dir = '/Volumes/mackeylab/Mackeylab/Individual_Folders/YiyuWang/HEAL_EEG/Scalp/'

for session_id in range(5):
    if session_id == 0:
        df = pd.read_csv(eeg_dir + f'mergedScalpEEG_REC_REO_0{session_id}.csv')
        # replace "_REC_0{session_id}" with empty string in column names to keep column names consistent across sessions
        df.columns = [col.replace(f'_REC_0{session_id}', '') for col in df.columns]
        df['session'] = 'baseline'

    else:
        ses_df = pd.read_csv(eeg_dir + f'mergedScalpEEG_REC_REO_0{session_id}.csv')
        ses_df.columns = [col.replace(f'_REC_0{session_id}', '') for col in ses_df.columns]
        df = pd.concat([df, ses_df])
        
        if session_id == 4:
            df['session'] = 'month6'
        else:     
            df['session'] = f'month{session_id}'
df.rename(columns={'subNum': 'study_id'}, inplace=True)   
# move session to the second column
df = df[['study_id', 'session'] + [col for col in df.columns if col != 'study_id' and col != 'session']]

df.to_csv(f'HEAL_EEG_scalp.csv', index=False)   

In [10]:
parcellation = 'schaefer400'
parcellation_source_dir =f'/Users/yiyuwang/Downloads/{parcellation}'

# combine all participants:

parcellation_files = glob.glob(parcellation_source_dir + f'/sub-*_task-rest_run-1_parcellation-{parcellation}.csv')
parcellation_files = np.sort(parcellation_files)
# for f in parcellation_files:
#     os.system(f'cp -r {f} {parcellation_source_dir}/task-rest_run-1/')


import pandas as pd

# Concatenate the CSV files into a big dataframe
for i, file in enumerate(parcellation_files):
    sub_df = pd.read_csv(file)
    sub_id = file.split('sub-')[1].split('_task')[0]
    sub_df['study_id'] = FormatSubID(sub_id)
    if i == 0:
        df = sub_df
    else:    
        df = pd.concat([df, sub_df])
# drop unnamed columns
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

# move study_id to the first column
df = df[['study_id'] + [col for col in df.columns if col != 'study_id']]
# drop the first column
# Save the big dataframe as a new CSV file
df.to_csv(f'HEAL_MRI_task-rest_parcellation-{parcellation}.csv', index=False)

In [23]:
# create parcellation for pressure glm:
import pandas as pd
import glob, sys
import nibabel as nib
import numpy as np
import os
from nilearn.connectome import ConnectivityMeasure
from nilearn import datasets
from nilearn.input_data import NiftiMasker, NiftiLabelsMasker, NiftiMapsMasker
import warnings
data_dir = '/Volumes/mackeylab/Mackeylab/Individual_Folders/Kaisa/fmri_results/univariate/model1_stimulation/1stlvl'
parcellation = 'schaefer400'

mask_img = f'{project_dir}/masks/tpl-MNI152NLin6Asym_res-02_desc-brain_mask.nii'



ROI_scheme = 'schaefer'
NROI = 200
if ROI_scheme == 'schaefer':
    NROI = int(NROI) # NROI will not be used if ROI_scheme is msdl
    ROI_scheme = f'schaefer{NROI}'
print(f'using ROI scheme - {ROI_scheme}')

if ROI_scheme == 'msdl':
    atlas = datasets.fetch_atlas_msdl()
    masker = NiftiMapsMasker(
        maps_img=atlas['maps'],
        resampling_target="mask",
        t_r=0.8,
        detrend=True,
        memory=f'{project_dir}/nilearn',
        mask_img = mask_img,
        standardize="zscore_sample",
        standardize_confounds="zscore_sample",
        verbose=0
    )
elif 'schaefer' in ROI_scheme:
    atlas = datasets.fetch_atlas_schaefer_2018(n_rois=NROI,resolution_mm=2)
    masker = NiftiLabelsMasker(
        labels_img=atlas['maps'],
        labels=atlas['labels'],
        t_r = 0.8,
        detrend=True,
        standardize=True,
        memory=f'{project_dir}/nilearn',
        mask_img = mask_img,
        verbose=0)

save_dir = f'/Users/yiyuwang/Desktop/SNAPL/Projects/HEAL_prediction/Data/cleaned_data'
file_list = glob.glob(f'{data_dir}/*/sub-*_stat-z_reg-stimulation_gm_masked.nii.gz')
save_file_path = save_dir + f'/HEAL_MRI_task-pressure_run-1_desc-pain_parcellation-{ROI_scheme}.csv'
run = 1
for i, file in enumerate(file_list):
    
    file_name = os.path.basename(file)
    s = file_name.split(f'sub-')[1].split('_stat-z')[0]
    
    
    if os.path.exists(file):
        fmri_img = nib.load(glob.glob(file)[0])
        parcellated_data = masker.fit_transform(fmri_img)

        # save parcellation to df
        parcellated_data_df = pd.DataFrame(parcellated_data,columns=atlas['labels'])

        # concatenate the parcellated data with the study_id
        parcellated_data_df['subject_id'] = FormatSubID(s)
        parcellated_data_df = parcellated_data_df[['subject_id'] + [col for col in parcellated_data_df.columns if col != 'subject_id']]

        if i == 0:
            df = parcellated_data_df
        else:
            df = pd.concat([df, parcellated_data_df])    

    else:
        
        print(f"{file} func file does not exist!")
        continue

df.to_csv(save_file_path, index=False)    

using ROI scheme - schaefer200


In [5]:
len_list, file_list = [], []
for session in ['baseline', 'month1', 'month2', 'month3', 'month6']:
    actigraphy_dir = f'/Volumes/Projects/HEAL_dataset_dev/ses-{session}/Actigraphy/'
    actigraphy_files = glob.glob(actigraphy_dir + 'sub-*.csv')
    for i, file in enumerate(actigraphy_files):
        sub_df = pd.read_csv(file)
        file_len = len(sub_df)
        len_list.append(file_len)
        file_list.append(file)
        if file_len != 20161:
            print(file, file_len)



/Volumes/Projects/HEAL_dataset_dev/ses-month1/Actigraphy/sub-bio0045_ses-month1_desc-actigraphy.csv 20092
/Volumes/Projects/HEAL_dataset_dev/ses-month1/Actigraphy/sub-bio0037_ses-month1_desc-actigraphy.csv 20084
/Volumes/Projects/HEAL_dataset_dev/ses-month2/Actigraphy/sub-bio0020_ses-month2_desc-actigraphy.csv 20155
/Volumes/Projects/HEAL_dataset_dev/ses-month2/Actigraphy/sub-bio0047_ses-month2_desc-actigraphy.csv 20091
/Volumes/Projects/HEAL_dataset_dev/ses-month2/Actigraphy/sub-bio0038_ses-month2_desc-actigraphy.csv 20084
/Volumes/Projects/HEAL_dataset_dev/ses-month3/Actigraphy/sub-bio0155_ses-month3_desc-actigraphy.csv 20341
/Volumes/Projects/HEAL_dataset_dev/ses-month3/Actigraphy/sub-bio0022_ses-month3_desc-actigraphy.csv 20192
/Volumes/Projects/HEAL_dataset_dev/ses-month3/Actigraphy/sub-bio0152_ses-month3_desc-actigraphy.csv 20198
/Volumes/Projects/HEAL_dataset_dev/ses-month3/Actigraphy/sub-bio0161_ses-month3_desc-actigraphy.csv 20206
/Volumes/Projects/HEAL_dataset_dev/ses-month3/

In [11]:

# concatenate all actigraphy data:
for session in ['baseline', 'month1', 'month2', 'month3', 'month6']:
    actigraphy_dir = f'/Volumes/Projects/HEAL_dataset_dev/ses-{session}/Actigraphy/'
    actigraphy_files = glob.glob(actigraphy_dir + 'sub-*.csv')
    actigraphy_files = np.sort(actigraphy_files)
    print(session)
    for i, file in enumerate(actigraphy_files):
        sub_df = pd.read_csv(file)
        sub_id = file.split('sub-')[1].split(f'_ses-{session}')[0]
        sub_df['study_id'] = FormatSubID(sub_id)
        sub_df['session'] = session
        if sub_df.shape[0] > 20161:
            sub_df = sub_df.iloc[:20161]
            print(f'{file}, {sub_df.shape[0]}')
        
        if i == 0:
            df = sub_df
        else:    
            df = pd.concat([df, sub_df])
    print(f"saving {session}")
    df.to_csv(f'HEAL_actigraphy_ses-{session}_timeseries.csv', index=False)
    print(len(df))       

baseline
saving baseline
2358837
month1
saving month1
2237725
month2
saving month2
2298201
month3
/Volumes/Projects/HEAL_dataset_dev/ses-month3/Actigraphy/sub-bio0022_ses-month3_desc-actigraphy.csv, 20161
/Volumes/Projects/HEAL_dataset_dev/ses-month3/Actigraphy/sub-bio0023_ses-month3_desc-actigraphy.csv, 20161
/Volumes/Projects/HEAL_dataset_dev/ses-month3/Actigraphy/sub-bio0142_ses-month3_desc-actigraphy.csv, 20161
/Volumes/Projects/HEAL_dataset_dev/ses-month3/Actigraphy/sub-bio0152_ses-month3_desc-actigraphy.csv, 20161
/Volumes/Projects/HEAL_dataset_dev/ses-month3/Actigraphy/sub-bio0155_ses-month3_desc-actigraphy.csv, 20161
/Volumes/Projects/HEAL_dataset_dev/ses-month3/Actigraphy/sub-bio0159_ses-month3_desc-actigraphy.csv, 20161
/Volumes/Projects/HEAL_dataset_dev/ses-month3/Actigraphy/sub-bio0161_ses-month3_desc-actigraphy.csv, 20161
/Volumes/Projects/HEAL_dataset_dev/ses-month3/Actigraphy/sub-bio0205_ses-month3_desc-actigraphy.csv, 20161
saving month3
2298354
month6
/Volumes/Projects

In [15]:
test = pd.read_csv('HEAL_actigraphy_ses-baseline_timeseries.csv')
print(np.unique(test.study_id).shape)
test = pd.read_csv('HEAL_actigraphy_ses-month1_timeseries.csv')
print(np.unique(test.study_id).shape)
test = pd.read_csv('HEAL_actigraphy_ses-month2_timeseries.csv')
print(np.unique(test.study_id).shape)
test = pd.read_csv('HEAL_actigraphy_ses-month3_timeseries.csv')
print(np.unique(test.study_id).shape)
test = pd.read_csv('HEAL_actigraphy_ses-month6_timeseries.csv')
print(np.unique(test.study_id).shape)

(117,)
(111,)
(114,)
(114,)
(115,)


In [15]:

# concatenate all actigraphy data:
counter = 0
for session in ['baseline', 'month1', 'month2', 'month3', 'month6']:
    actigraphy_dir = f'/Volumes/Projects/HEAL_dataset_dev/ses-{session}/Actigraphy/'
    actigraphy_files = glob.glob(actigraphy_dir + 'sub-*.csv')
    actigraphy_files = np.sort(actigraphy_files)
    print(session)
    for i, file in enumerate(actigraphy_files):
        sub_df = pd.read_csv(file)
        sub_id = file.split('sub-')[1].split(f'_ses-{session}')[0]
        sub_df['study_id'] = FormatSubID(sub_id)
        sub_df['session'] = session
        counter += 1
    
        #if sub_df.shape[0] != 20161:
        #    print(f'{file}, {sub_df.shape[0]}')
        #sub_file_save_path = f'/Users/yiyuwang/Library/CloudStorage/OneDrive-Stanford/HEAL_Actigraphy/ses-{session}/sub-{sub_id}_ses-{session}_actigraphy.csv'
        #sub_df.to_csv(sub_file_save_path, index=False)  
    print(f"{session} {counter}")
print(counter)        #        

baseline
month1
month2
month3
month6
571


# ------------------------- end here 2024/06/03 -------------------------

In [6]:
# list of columns to keep:

col_to_keep = ['subject_id', 'redcap_event_name', 
                'PainInT', 'DepT',	'AnxT',	'AngT',	'FatigeT', 'SleepDisT',	'SleepImpT', 'IsolationT', 'eSupportT',	'PhyFxT',
               'cpaq_total', 'cpaq_willingness', 'cpaq_activity_engagement', 
               'pcs_score_rumination',	'pcs_score_magnification',	'pcs_score_helplessness', 'pcs_score_total',
               'scltotal', 'pseqtotal', 'gadtotal', 'phqtotal', 'tsktotal',	
               'sleephrs', 'HICP3','pain_frequency', 'life_limitation',	'describe_pain_number',	'interferes_enjoyment',	'interferes_general_activity',	'working_not_working',
               'taps_1', 'taps_2', 'taps_3', 'taps_4',	'taps_5',
               'ppiy_1', 'ppiy_2a', 'ppiy_3a']


# drop_cols = [
#         'pain_frequency', 
#         'life_limitation',
#         'describe_pain_number',
#         'interferes_enjoyment',
#         'interferes_general_activity', 
#         'working_not_working',
#         'HICP3',
#         'PainInT'
# ]

# col_to_keep = [x for x in col_to_keep if x not in drop_cols]
print(col_to_keep)

['subject_id', 'redcap_event_name', 'PainInT', 'DepT', 'AnxT', 'AngT', 'FatigeT', 'SleepDisT', 'SleepImpT', 'IsolationT', 'eSupportT', 'PhyFxT', 'cpaq_total', 'cpaq_willingness', 'cpaq_activity_engagement', 'pcs_score_rumination', 'pcs_score_magnification', 'pcs_score_helplessness', 'pcs_score_total', 'scltotal', 'pseqtotal', 'gadtotal', 'phqtotal', 'tsktotal', 'sleephrs', 'HICP3', 'pain_frequency', 'life_limitation', 'describe_pain_number', 'interferes_enjoyment', 'interferes_general_activity', 'working_not_working', 'taps_1', 'taps_2', 'taps_3', 'taps_4', 'taps_5', 'ppiy_1', 'ppiy_2a', 'ppiy_3a']


In [7]:
pro_df['subject_id'] = pro_df['study_id'].apply(FormatSubID)
pro_df_cleaned = pro_df[col_to_keep]

pro_df_cleaned = pro_df_cleaned.rename(columns={'redcap_event_name':'session'})
pro_df_cleaned['session'] = pro_df_cleaned['session'].replace({'baseline_arm_1': 'baseline', '1_month_followup_arm_1': 'month1', '2_month_followup_arm_1': 'month2', '3_month_followup_arm_1': 'month3', '6_month_followup_arm_1': 'month6'})
# pro_df_cleaned = pro_df_cleaned.recode({'session': {'baseline_visit_arm_1': 'baseline', '1_month_followup_arm_1': '1month', '2_month_followup_arm_1': '2month', '3_month_followup_arm_1': '3month', '6_month_followup_arm_1': '6month'}})

pro_head = pro_df_cleaned[['subject_id', 'session']]
pro_df_cleaned_data = pro_df_cleaned.copy().drop(['subject_id', 'session'], axis=1).apply(pd.to_numeric, errors='coerce')
pro_df_cleaned_data['sleephrs'] = pro_df_cleaned_data['sleephrs'].apply(lambda x: np.nan if x > 24 else x)
pro_df_final = pro_head.join(pro_df_cleaned_data)
pro_df_final.head()





,subject_id,session,PainInT,DepT,AnxT,AngT,FatigeT,SleepDisT,SleepImpT,IsolationT,...,interferes_general_activity,working_not_working,taps_1,taps_2,taps_3,taps_4,taps_5,ppiy_1,ppiy_2a,ppiy_3a
0,bio0001,baseline,73.2,51.2,52.5,49.8,66.1,63.2,64.9,52.6,...,10.0,1.0,4.0,4.0,4.0,4.0,4.0,8.0,5.0,6.0
1,bio0001,month1,71.7,47.6,45.9,48.9,67.4,61.6,68.0,48.1,...,8.0,1.0,4.0,4.0,4.0,4.0,4.0,9.0,6.0,4.0
2,bio0001,month2,71.7,46.3,47.3,45.8,70.5,62.7,65.9,48.9,...,9.0,1.0,4.0,4.0,4.0,4.0,4.0,9.0,6.0,6.0
3,bio0001,month3,71.4,38.2,51.9,47.3,74.2,66.2,66.2,48.3,...,8.0,1.0,4.0,4.0,4.0,4.0,4.0,8.0,6.0,6.0
4,bio0001,month6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.0,1.0,4.0,4.0,4.0,4.0,4.0,10.0,6.0,5.0


In [8]:
# save the cleaned data, one csv for each session
target_dir = '/Volumes/Projects/HEAL_dataset_dev'
for session in pro_df_final['session'].unique():
    pro_df_final[pro_df_final['session']==session].to_csv(f'{target_dir}/ses-{session}/PROs/ses-{session}_desc-PRO.csv', index=False)

In [9]:

threshold = 65
threshold_str = 65

y_df = pro_df_final[['subject_id', 'session', 'PainInT']]

y_df['ImpactGroup'] = y_df['PainInT'].apply(lambda x: 1 if x and x >= threshold else 0 if x else None)
for session in y_df['session'].unique():
    y_df[y_df['session']==session].to_csv(f'{target_dir}/ses-{session}/PROs/ses-{session}_desc-PainInT_threshold-{threshold}.csv', index=False)


In [10]:
def convert_to_year(number):
    number = str(number).split('/')[2]
    if len(number) != 2:
        raise ValueError("Input must be a two-digit number")
    try:
        year = int(number)
    except ValueError:
        raise ValueError("Input must be a valid number")
    if year < 0 or year > 99:
        raise ValueError("Input must be between 00 and 99")
    if year < 25:
        return 2000 + year
    return 1900 + year



# Example usage
input_number = "6/8/02"
converted_number = convert_to_year(input_number)
print(converted_number)


2002


In [11]:

demographic_df = pd.read_csv(f'raw_data/PROS/demographics_oncore.csv', skiprows=4, index_col=0)
demographic_df['StudyYear'] = demographic_df['On Study Date'].apply(convert_to_year)
demographic_df['BirthYear'] =demographic_df['Birth Date'].apply(convert_to_year)


demographic_df['age'] = demographic_df['StudyYear'] - demographic_df['BirthYear']
demographic_df['subject_id'] = demographic_df['Sequence No.'].apply(FormatSubID)
demographic_df = demographic_df.rename(columns={'Biological Sex': 'sex'})
demographic_cleaned_df = demographic_df[['subject_id', 'age', 'sex', 'Race', "Ethnicity"]]
demographic_cleaned_df.sort_values('subject_id', inplace=True)
demographic_cleaned_df.to_csv(f'{target_dir}/participants_demographics.csv', index=False)